In [52]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Пагинация

In [53]:
BASE_LIST_URL = "https://lifehacker.ru/topics/technology/"

 # Задание 1 - Определить формат ссылки для пагинации
 Формируем URL для страницы рубрики
 Формат ссылки для пагинации:
      1-я страница: /topics/technology/
      далее: /topics/technology/?page=N

In [54]:
def make_page_url(page: int) -> str:
    if page == 1:
        return BASE_URL
    return f"{BASE_URL}?page={page}"

#     Задание 2 - В разметке страницы найти уникальные классы идентификаторы блока с названием и содержанием материала

    Собираем ссылки на статьи с 10 страниц
    Функция get_article_links_from_page возвращает список ссылок на статьи с одной страницы рубрики

    Получаем HTML страницы рубрики, находим все article-карточки
    и вытаскиваем из них ссылки на материалы.

In [46]:
def get_article_links_from_page(page: int) -> list[str]:
    url = make_page_url(page)
    print(f"[PAGE {page}] GET {url}")

    resp = requests.get(url, timeout=10)
    resp.raise_for_status()
    resp.encoding = "utf-8"

    soup = BeautifulSoup(resp.text, "lxml")

    page_links: list[str] = []

    # 1. Находим общий контейнер со статьями
    root = soup.find("div", class_="article-card-container")
    inner = root.find("div", class_="container") if root else None

    if not (root and inner):
        print("  ERROR: Не найдены article-card-container/container — проверь структуру сайта.")
        return page_links

    # 2. Карточки статей
    wrappers = inner.find_all("div", class_="article-card__small-wrapper")

    for wrapper in wrappers:
        # 3. Ссылка внутри карточки
        a = wrapper.find("a", class_="lh-small-article-card__link", href=True)
        if not a:
            continue

        href = a["href"]

        # относительные → абсолютные
        if href.startswith("/"):
            href = "https://lifehacker.ru" + href

        # фильтрация
        if (
            href.startswith("https://lifehacker.ru/")
            and "/topics/" not in href
            and "/tag/" not in href
            and "#" not in href
            and "?" not in href
        ):
            if href not in page_links:
                page_links.append(href)

    print(f"  найдено ссылок на странице: {len(page_links)}")
    return page_links


# Задание 3 - Получить содержимое десяти страниц списка материалов, выделить из него ссылки на каждый материал
Собираем ссылки с первых 10 страниц

In [48]:
all_urls = []
seen = set()

for page in range(1, 11):
    page_links = get_article_links_from_page(page)
    for url in page_links:
        if url not in seen:
            seen.add(url)
            all_urls.append(url)

print(f"\nИтого уникальных ссылок: {len(all_urls)}")
print("\nПервые 10 ссылок:")
for link in all_urls[:10]:
    print(" •", link)


[PAGE 1] GET https://lifehacker.ru/topics/technology/
  найдено ссылок на странице: 30
[PAGE 2] GET https://lifehacker.ru/topics/technology/?page=2
  найдено ссылок на странице: 30
[PAGE 3] GET https://lifehacker.ru/topics/technology/?page=3
  найдено ссылок на странице: 30
[PAGE 4] GET https://lifehacker.ru/topics/technology/?page=4
  найдено ссылок на странице: 30
[PAGE 5] GET https://lifehacker.ru/topics/technology/?page=5
  найдено ссылок на странице: 30
[PAGE 6] GET https://lifehacker.ru/topics/technology/?page=6
  найдено ссылок на странице: 30
[PAGE 7] GET https://lifehacker.ru/topics/technology/?page=7
  найдено ссылок на странице: 30
[PAGE 8] GET https://lifehacker.ru/topics/technology/?page=8
  найдено ссылок на странице: 30
[PAGE 9] GET https://lifehacker.ru/topics/technology/?page=9
  найдено ссылок на странице: 30
[PAGE 10] GET https://lifehacker.ru/topics/technology/?page=10
  найдено ссылок на странице: 30

Итого уникальных ссылок: 300

Первые 10 ссылок:
 • https://lifeh

# Задание 5 - Распарсить полученный текст разметки с помощью BeautifulSoup, вытащив по каждой ссылке заголовок и содержание материала
Обходим ссылки и парсим статьи (html + BS4)

По ссылке на материал:
      - получаем html (requests)
      - парсим (BeautifulSoup)
      - вытаскиваем заголовок и содержимое

In [49]:
def parse_article(url: str) -> dict:
    resp = requests.get(url, headers=HEADERS, timeout=10)
    resp.raise_for_status()
    resp.encoding = "utf-8"

    soup = BeautifulSoup(resp.text, "lxml")


    h1 = soup.find("h1", class_="article-card__title")
    title = h1.get_text(strip=True) if h1 else "NO_TITLE"

    text = ""

    article_tag = soup.find(
        "article",
        id="article"
    )

    if article_tag:
        paragraphs = [
            p.get_text(" ", strip=True)
            for p in article_tag.find_all("p")
        ]
        text = "\n".join(p for p in paragraphs if p)


    if not text:
        paragraphs = [
            p.get_text(" ", strip=True)
            for p in soup.find_all("p")
        ]
        text = "\n".join(p for p in paragraphs if p)

    if not text:
        text = "NO_TEXT"

    return {
        "url": url,
        "title": title,
        "text": text,
    }


# Задание 4 - Перебрать все полученные ссылки и получить html-код каждого материала

In [50]:
rows = []

total = len(all_urls)
for i, url in enumerate(all_urls, start=1):
    print(f"[{i}/{total}] Парсим: {url}")
    try:
        rows.append(parse_article(url))
    except Exception as e:
        print(f"Ошибка при обработке {url}: {e}")


[1/300] Парсим: https://lifehacker.ru/xarakteristiki-galaxy-s26-ultra/
[2/300] Парсим: https://lifehacker.ru/anons-modretro-m64/
[3/300] Парсим: https://lifehacker.ru/igry-android-ios-noyabr-2025/
[4/300] Парсим: https://lifehacker.ru/anons-black-shark-gs3-ultra/
[5/300] Парсим: https://lifehacker.ru/zamedlenie-whatsapp/
[6/300] Парсим: https://lifehacker.ru/tysyachi-rekomendacii-po-naushnikam-s-reddit/
[7/300] Парсим: https://lifehacker.ru/prilozhenija-android-noyabr-2025/
[8/300] Парсим: https://lifehacker.ru/energoeffektivnyi-rezhim-v-google-kartax/
[9/300] Парсим: https://lifehacker.ru/20-gift-gadgets/
[10/300] Парсим: https://lifehacker.ru/mask-architects-solaris/
[11/300] Парсим: https://lifehacker.ru/google-ogranichila-besplatnyi-dostup-k-gemini-3-pro/
[12/300] Парсим: https://lifehacker.ru/anons-reetle-smartink-i/
[13/300] Парсим: https://lifehacker.ru/predstavlena-klaviatura-kernelcom/
[14/300] Парсим: https://lifehacker.ru/top-chat-botov-2025/
[15/300] Парсим: https://lifehac

# Задание 6 - Создать датайфрейм с полученными данными

In [51]:
df = pd.DataFrame(rows, columns=["url", "title", "text"])
df.to_csv("lifehacker_technology_10_pages.csv", index=False)

df.head()


,url,title,text
0,https://lifehacker.ru/xarakteristiki-galaxy-s2...,Инсайдер сравнил характеристики смартфонов сер...,"Инсайдер Ice Universe, известный по утечкам о ..."
1,https://lifehacker.ru/anons-modretro-m64/,Представлен прозрачный аналог Nintendo 64 с 4K...,Компания ModRetro представила ретро-консоль M6...
2,https://lifehacker.ru/igry-android-ios-noyabr-...,12 новых игр для Android и iOS: лучшее за ноябрь,Продолжаем дайджест мобильных новинок. В этом ...
3,https://lifehacker.ru/anons-black-shark-gs3-ul...,Xiaomi показала сверхзащищённые часы Black Sha...,Xiaomi представила новые смарт-часы под брендо...
4,https://lifehacker.ru/zamedlenie-whatsapp/,В РКН подтвердили замедление WhatsApp: возможн...,Роскомнадзор подтвердил замедление работы What...
